In [ ]:
import pandas as pd
import re
import os
print(os.listdir("../input"))
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, '', raw_html)

In [ ]:
def removeCharacters(text):
    bannedChars = ["\n","\r","\t","!","?"]
    for char in bannedChars:
        while char in text:
            text = text.replace(char, " ")
    return text

In [ ]:
q = pd.read_csv('../input/Questions.csv', encoding='ISO-8859-1')
#Get only relevant columns
q = q[["Id", "Score", "Title", "Body"]]
#Remove html tags
q['Body'] = q['Body'].apply(cleanhtml)
#Join Title and body
q["Title"] = q["Title"].map(str) + " " + q["Body"]
#Delete redundant body
q = q[["Id", "Score", "Title"]]

In [ ]:
#Rename Title
q = q.rename(index=str, columns={"Title": "Question"})
#Set index to id
q = q.set_index('Id')
q.head()

In [ ]:
#Remove bad questions
q = q[q["Score"] > -1]
#Remove invalid characters
q["Question"] = q["Question"].apply(removeCharacters)
q.head()

# Answers

In [ ]:
a = pd.read_csv('../input/Answers.csv', encoding='ISO-8859-1')
#Keep only relevant columns
a = a[["ParentId","Score","Body"]]

In [ ]:
#remove html
a['Body'] = a['Body'].apply(cleanhtml)
#remove invalid characters
a['Body'] = a['Body'].apply(removeCharacters)
a.head()

In [ ]:
def getFunctions(text):
    words = text.split(" ")
    functions = ""
    for word in words:
        if "(" in word and ")" in word:
            functions += word +","
    return functions[0:len(functions)-2]

In [ ]:
def getURLs(text):
    urls = re.findall("(?P<url>https?://[^\s]+)", text)
    urlText = ""
    for url in urls:
        while True:
            if not url[len(url)-1].isdigit() and not url[len(url)-1].isalpha():
                url = url[0:len(url)-2]
            else:
                break
        urlText += url+","
    return urlText[0:len(urlText)-2]

In [ ]:
a['URL'] = a['Body'].apply(getURLs)
a['Function'] = a['Body'].apply(getFunctions)
a.head()

# Delete bad answers, delete answers without questions and visa versa

In [ ]:
a = a[a["Score"] > -1]

#check all questions have an answer
q = q[q.index.isin(a["ParentId"])]
q.head()

In [ ]:
#check answer has a question
a = a[a["ParentId"].isin(q.index)]
a.head()

In [ ]:
def toLower(text):
    return text.lower()

In [ ]:
q["Question"] = q["Question"].apply(toLower)
a["Body"] = a["Body"].apply(toLower)

In [ ]:
stop_words = set(stopwords.words('english'))
def removeStopWords(text):
    words = text.split(" ")
    txt = ""
    for word in words:
        if word not in stop_words:
            txt += word + " "
    return txt

In [ ]:
q["Question"] = q["Question"].apply(removeStopWords)
q.head()

In [ ]:
a["Body"] = a["Body"].apply(removeStopWords)
a.head()

In [ ]:
tokenizer = Tokenizer()
answers = []
questions = []

In [ ]:
tokenizer.fit_on_texts(q["Question"].values)
X = tokenizer.texts_to_sequences(q["Question"].values)

In [ ]:
X = pad_sequences(X, maxlen=1000)